THIS IS CODE FOR CALULATING SLOUCH BASED ON THE ANGLE BETWEEN YOUR LEFT SHOULDER RIGHT SHOULDER AND HIP AND VIS VERSA

In [5]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize webcam and mediapipe
cap = cv2.VideoCapture(0)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def capture_reference_angles():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate reference angles
            left_angle = calculate_angle(left_shoulder, right_shoulder, right_hip)
            right_angle = calculate_angle(right_shoulder, left_shoulder, left_hip)

            # Display angles on the frame
            cv2.putText(frame, f"Left Angle: {int(left_angle)}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Angle: {int(right_angle)}", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Capture Reference Posture', frame)

            if cv2.waitKey(1) & 0xFF == ord('s'):
                # Save the reference angles
                cv2.imwrite('reference_posture.png', frame)
                cv2.destroyAllWindows()
                cap.release()
                return left_angle, right_angle

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

reference_left_angle, reference_right_angle = capture_reference_angles()


In [7]:
import cv2
import mediapipe as mp
import numpy as np

# Reinitialize webcam and mediapipe
cap = cv2.VideoCapture(0)
pose = mp_pose.Pose()

def check_posture(reference_left_angle, reference_right_angle, tolerance=10):
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate current angles
            current_left_angle = calculate_angle(left_shoulder, right_shoulder, right_hip)
            current_right_angle = calculate_angle(right_shoulder, left_shoulder, left_hip)

            # Check if slouching
            if (abs(current_left_angle - reference_left_angle) > tolerance or
                abs(current_right_angle - reference_right_angle) > tolerance):
                cv2.putText(frame, "You are slouching!", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, "Good Posture", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Display current angles on the frame
            cv2.putText(frame, f"Left Angle: {int(current_left_angle)}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Angle: {int(current_right_angle)}", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Posture Monitoring', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

check_posture(reference_left_angle, reference_right_angle)


THE FOLLOWING CODE WILL BE TO CHECK IF THE Z COORDINATES OF YOUR LEFT AND RIGHT SHOULDER HAVE GONE BELOW THE LIMIT INDICATING THAT U HAVE SLOUCHED

In [20]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize webcam and mediapipe
cap = cv2.VideoCapture(0)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def capture_reference_heights():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

            cv2.putText(frame, "Press 's' to save reference position", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Capture Reference Posture', frame)

            if cv2.waitKey(1) & 0xFF == ord('s'):
                # Save the reference y-coordinates
                cv2.imwrite('reference_posture_y.png', frame)
                cv2.destroyAllWindows()
                cap.release()
                return left_shoulder_y, right_shoulder_y

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Capture the reference shoulder heights using the y-coordinate
reference_left_shoulder_y, reference_right_shoulder_y = capture_reference_heights()


In [21]:
import cv2
import mediapipe as mp
import numpy as np

# Reinitialize webcam and mediapipe
cap = cv2.VideoCapture(0)
pose = mp_pose.Pose()

def check_posture(reference_left_shoulder_y, reference_right_shoulder_y, tolerance=0.07):  # 7 cm tolerance
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            current_left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            current_right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

            # Check if slouching based on y-coordinate (vertical height)
            if (current_left_shoulder_y > reference_left_shoulder_y + tolerance or
                current_right_shoulder_y > reference_right_shoulder_y + tolerance):
                cv2.putText(frame, "You are slouching!", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, "Good Posture", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Display current y-coordinates on the frame
            cv2.putText(frame, f"Left Shoulder Y: {round(current_left_shoulder_y, 4)}", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Shoulder Y: {round(current_right_shoulder_y, 4)}", (10, 110), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Posture Monitoring', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Monitor the posture using the y-coordinate
check_posture(reference_left_shoulder_y, reference_right_shoulder_y, tolerance=0.07)
